In [5]:
import numpy as np
import pandas as pd
import plotly.express as px
#import missingno
import os
import glob

from __future__ import print_function

#import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
from multiprocessing import Pool, cpu_count

import polars as pl
#from utils import getConnection

%matplotlib inline

In [16]:
df = pl.read_csv('/Users/cgarciay/Desktop/Laval_Master_Computer/research/MIMIC/ADMISSIONS.csv.gz')
ids = pl.read_csv("/Users/cgarciay/Desktop/Laval_Master_Computer/research/ids_complete.csv")
df_filtered = df.filter(pl.col('SUBJECT_ID').is_in( ids["0"]))


In [22]:

ruta_archivos = 'data/RAW/MIMIC/*.csv.gz'  # Puedes cambiar '*.csv' por la extensión que desees

# Lee todos los archivos con la extensión especificada
archivos = glob.glob(ruta_archivos)
archivos
 

['/Users/cgarciay/Desktop/Laval_Master_Computer/research/MIMIC/PATIENTS.csv.gz',
 '/Users/cgarciay/Desktop/Laval_Master_Computer/research/MIMIC/DIAGNOSES_ICD.csv.gz',
 '/Users/cgarciay/Desktop/Laval_Master_Computer/research/MIMIC/PRESCRIPTIONS.csv.gz',
 '/Users/cgarciay/Desktop/Laval_Master_Computer/research/MIMIC/D_ICD_DIAGNOSES.csv.gz',
 '/Users/cgarciay/Desktop/Laval_Master_Computer/research/MIMIC/ADMISSIONS.csv.gz',
 '/Users/cgarciay/Desktop/Laval_Master_Computer/research/MIMIC/PROCEDURES_ICD.csv.gz',
 '/Users/cgarciay/Desktop/Laval_Master_Computer/research/MIMIC/D_ICD_PROCEDURES.csv.gz']

In [15]:
#from itertools import pairwise

def multidimensional_to_numpy(s):
    dimensions = [1, len(s)]
    while s.dtype == pl.List:
        s = s.explode()
        dimensions.append(len(s))
    dimensions = [p[1] // p[0] for p in pairwise_distances(dimensions)]
    return s.to_numpy().reshape(dimensions)

In [55]:
def list_(x):
    if len(x) ==len_c:
        pass
    else:
        num = max(len_c)-len(x)
       
        l = [ list(itertools.repeat(0,max_por_lista )) for i in range(num)]

        result = x.to_list() + l
    return result

In [94]:
def temporal(x):
    if len(x) ==len_c:
        vec = list(itertools.repeat(1,len_c ))
    else:
        num = max(len_c)-len(x)
        vec = list(itertools.repeat(1,len(x) ))
        l = list(itertools.repeat(0,num )) 

        vec = vec + l
    return vec

In [76]:
def zeors_lista(x):
    return  [i.to_list()+list(itertools.repeat(0,max_por_lista-len(i) )) for i in x if len(i)!=max_por_lista]

# Filtered

In [34]:
a = [i for i in archivos if "D_ICD_" not in i] 
a

['/Users/cgarciay/Desktop/Laval_Master_Computer/research/MIMIC/PATIENTS.csv.gz',
 '/Users/cgarciay/Desktop/Laval_Master_Computer/research/MIMIC/DIAGNOSES_ICD.csv.gz',
 '/Users/cgarciay/Desktop/Laval_Master_Computer/research/MIMIC/PRESCRIPTIONS.csv.gz',
 '/Users/cgarciay/Desktop/Laval_Master_Computer/research/MIMIC/ADMISSIONS.csv.gz',
 '/Users/cgarciay/Desktop/Laval_Master_Computer/research/MIMIC/PROCEDURES_ICD.csv.gz']

In [4]:
import numpy as np
import pandas as pd
import plotly.express as px
#import missingno
import os
import glob

from __future__ import print_function

#import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
from multiprocessing import Pool, cpu_count

import polars as pl
#from utils import getConnection

def concat_archivo_primeto(procedures,admi,df,ruta_archivos,save):
    df = pl.read_csv(admi)
    df_filtered = df
    
    df_filtered = df_filtered.with_columns(pl.col("SUBJECT_ID").cast(pl.Utf8))
    df_filtered = df_filtered.with_columns(pl.col("HADM_ID").cast(pl.Utf8))
    archivos = glob.glob(ruta_archivos)
    archivos
    df_filtered.shape
    unique_subid = []

    for i in archivos:
        aux =  pl.read_csv(i,infer_schema_length=0
                        )
    
        try:
            aux = aux.with_columns(pl.col("SUBJECT_ID").cast(pl.Utf8))
            aux = aux.with_columns(pl.col("HADM_ID").cast(pl.Utf8))
            aux = aux.groupby(['SUBJECT_ID','HADM_ID'], maintain_order=True).all()
            #NOTA  SE ELIMIA ICUSTARY_ID del archivo 'dataset/ICUSTAYS.csv.gz' ya que esta duplicado"ICUSTAY_ID"
            aux = aux.select(pl.exclude("ROW_ID"))
            if i == 'dataset/ICUSTAYS.csv.gz':
                aux = aux.select(pl.exclude("ICUSTAY_ID"))
                
            elif i =='procedures':    
                aux = aux.select(pl.exclude("SEQ_NUM"))
            df_filtered=df_filtered.join(aux, on=['SUBJECT_ID','HADM_ID'], how="left")

            #df_filtered = pd.merge(df_filtered, aux, on=['SUBJECT_ID','HADM_ID'], how='left')
            print("concat, "+i)
        except:
            
                
            aux = aux.with_columns(pl.col("SUBJECT_ID").cast(pl.Utf8))
            aux = aux.filter(pl.col('SUBJECT_ID').is_in( ids["0"]))
            #aux = aux.groupby(['SUBJECT_ID'], maintain_order=True).all()
            
            aux = aux.select(pl.exclude("ROW_ID"))
            df_filtered=df_filtered.join(aux, on=['SUBJECT_ID'], how="left")
            unique_subid.append(i)
            print(i)
            if save == True:
                df_filtered.write_csv('df_non_filtered.parquet')
        print(df_filtered)       
            #df_filtered = pd.merge(df_filtered, aux, on=['SUBJECT_ID'], how='left')
    return df_filtered        

In [9]:
ruta_actual = os.getcwd()
print(ruta_actual)

/Users/cgarciay/Desktop/Laval_Master_Computer/research/Synthetic-Data-Deep-Learning/preprocessing/notebook


In [11]:
ruta_actual = os.getcwd()
print(ruta_actual)

#RUTA = ruta_actual +"/Synthetic-Data-Deep-Learning/"
os.chdir('/Users/cgarciay/Desktop/Laval_Master_Computer/research/Synthetic-Data-Deep-Learning')

df = pl.read_csv('data/RAW/MIMIC/ADMISSIONS.csv.gz')
procedures = "data/RAW/MIMIC/ADMISSIONS.csv.gz"
admi = 'data/RAW/MIMIC/ADMISSIONS.csv.gz'

ruta_archivos = 's_data\*.csv.gz'  # Puedes cambiar '*.csv' por la extensión que desees
save = False #dfale
# Lee todos los archivos con la extensión especificada

df_filtered =concat_archivo_primeto(procedures,admi,df,ruta_archivos,save)
if save:
   df_filtered.write_parquet("df_non_filtered.parquet")


/Users/cgarciay/Desktop/Laval_Master_Computer/research/Synthetic-Data-Deep-Learning


# Original

In [ ]:


df_filtered = df_filtered.with_columns(pl.col("SUBJECT_ID").cast(pl.Utf8))
df_filtered = df_filtered.with_columns(pl.col("HADM_ID").cast(pl.Utf8))

df_filtered.shape

for i in a:
    aux =  pl.read_csv(i,infer_schema_length=0
                      )
 
    try:
        aux = aux.with_columns(pl.col("SUBJECT_ID").cast(pl.Utf8))
        aux = aux.with_columns(pl.col("HADM_ID").cast(pl.Utf8))
        aux = aux.groupby(['SUBJECT_ID','HADM_ID'], maintain_order=True).all()
        #NOTA  SE ELIMIA ICUSTARY_ID del archivo 'dataset/ICUSTAYS.csv.gz' ya que esta duplicado"ICUSTAY_ID"
        aux = aux.select(pl.exclude("ROW_ID"))
        if i == 'dataset/ICUSTAYS.csv.gz':
            aux = aux.select(pl.exclude("ICUSTAY_ID"))
            
        elif i ==procedures:    
            aux = aux.select(pl.exclude("SEQ_NUM"))
        df_filtered=df_filtered.join(aux, on=['SUBJECT_ID','HADM_ID'], how="left")

        #df_filtered = pd.merge(df_filtered, aux, on=['SUBJECT_ID','HADM_ID'], how='left')
        print("concat, "+i)
    except:
        
            
        aux = aux.with_columns(pl.col("SUBJECT_ID").cast(pl.Utf8))
        aux = aux.filter(pl.col('SUBJECT_ID').is_in( ids["0"]))
        #aux = aux.groupby(['SUBJECT_ID'], maintain_order=True).all()
        
        aux = aux.select(pl.exclude("ROW_ID"))
        df_filtered=df_filtered.join(aux, on=['SUBJECT_ID'], how="left")

        print(i)
        #df_filtered = pd.merge(df_filtered, aux, on=['SUBJECT_ID'], how='left')
        

In [125]:
#df_filtered.write_parquet("df_filtered.parquet")
df_filtered.write_parquet("df_non_filtered.parquet")


In [140]:
df_filtered

ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,COSTCENTER,CHARTDATE,CPT_CD,CPT_NUMBER,CPT_SUFFIX,TICKET_ID_SEQ,SECTIONHEADER,SUBSECTIONHEADER,DESCRIPTION,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,SEQ_NUM,ICD9_CODE,ICUSTAY_ID,…,DIAGNOSIS_right,HOSPITAL_EXPIRE_FLAG_right,HAS_CHARTEVENTS_DATA_right,SUBMIT_WARDID,SUBMIT_CAREUNIT,CURR_WARDID_right,CURR_CAREUNIT_right,CALLOUT_WARDID,CALLOUT_SERVICE,REQUEST_TELE,REQUEST_RESP,REQUEST_CDIFF,REQUEST_MRSA,REQUEST_VRE,CALLOUT_STATUS,CALLOUT_OUTCOME,DISCHARGE_WARDID,ACKNOWLEDGE_STATUS,CREATETIME,UPDATETIME,ACKNOWLEDGETIME,OUTCOMETIME,FIRSTRESERVATIONTIME,CURRENTRESERVATIONTIME,SEQ_NUM_right,ICD9_CODE_right,TRANSFERTIME,PREV_SERVICE,CURR_SERVICE,DBSOURCE_right,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME_right,OUTTIME_right,LOS_right
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str,str,str,str,str,list[str],list[str],list[str],…,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str]
22,"""23""","""152223""","""2153-09-03 07:…","""2153-09-08 19:…",null,"""ELECTIVE""","""PHYS REFERRAL/…","""HOME HEALTH CA…","""Medicare""",null,"""CATHOLIC""","""MARRIED""","""WHITE""",null,null,"""CORONARY ARTER…",0,1,null,null,null,null,null,null,null,null,null,"""M""","""2082-07-17 00:…",null,null,null,"""0""","[""1"", ""2"", … ""8""]","[""41401"", ""4111"", … ""3899""]","[""227807"", ""227807"", … null]",…,"[""CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS GRAFT/SDA""]","[""0""]","[""1""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""1"", ""2"", … ""7""]","[""3613"", ""3615"", … ""9907""]","[""2153-09-03 00:21:37""]",[null],"[""CSURG""]","[""carevue""]","[""CSRU""]","[""CSRU""]","[""14""]","[""14""]","[""2153-09-03 09:38:55""]","[""2153-09-04 15:59:11""]","[""1.2641""]"
23,"""23""","""124321""","""2157-10-18 19:…","""2157-10-25 14:…",null,"""EMERGENCY""","""TRANSFER FROM …","""HOME HEALTH CA…","""Medicare""","""ENGL""","""CATHOLIC""","""MARRIED""","""WHITE""",null,null,"""BRAIN MASS""",0,1,"[""ICU"", ""ICU"", … ""ICU""]","[null, null, … null]","[""99255"", ""99232"", … ""99254""]","[""99255"", ""99232"", … ""99254""]","[null, null, … null]","[""1"", ""2"", … ""4""]","[""Evaluation and management"", ""Evaluation and management"", … ""Evaluation and management""]","[""Consultations"", ""Hospital inpatient services"", … ""Consultations""]","[null, null, … null]","""M""","""2082-07-17 00:…",null,null,null,"""0""","[""1"", ""2"", … ""10""]","[""2252"", ""3485"", … ""V1582""]","[""234044"", ""234044"", … null]",…,"[""BRAIN MASS""]","[""0""]","[""1""]","[""57""]",[null],"[""31""]","[""SICU""]","[""31""]","[""NSURG""]","[""1""]","[""0""]","[""0""]","[""0""]","[""0""]","[""Inactive""]","[""Discharged""]","[""31""]","[""Acknowledged""]","[""2157-10-22 10:42:44""]","[""2157-10-22 10:42:44""]","[""2157-10-22 10:59:46""]","[""2157-10-22 16:25:07""]","[""2157-10-22 14:40:12""]",[null],"[""1""]","[""0151""]","[""2157-10-18 19:34:59""]",[null],"[""NSURG""]","[""metavision""]","[""SICU""]","[""SICU""]","[""57""]","[""57""]","[""2157-10-21 11:40:38""]","[""2157-10-22 16:08:48""]","[""1.1862""]"
33,"""34""","""115799""","""2186-07-18 16:…","""2186-07-20 16:…",null,"""EMERGENCY""","""TRANSFER FROM …","""HOME""","""Medicare""","""ENGL""","""CATHOLIC""","""MARRIED""","""WHITE""",null,null,"""CHEST PAIN\CAT…",0,1,"[""ICU"", ""ICU"", "

# Preprocesing Dataframe medicament

In [147]:
df = pl.read_csv("dataset/PRESCRIPTIONS.csv.gz",infer_schema_length=0)
ids = pl.read_csv("ids.csv")
df = df.filter(pl.col('SUBJECT_ID').is_in( ids["Subjects_id"]))


In [162]:
df.head()

ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""1389712""","""21""","""109451""","""217847""","""2134-09-13 00:…","""2134-09-15 00:…","""BASE""","""Vial""",null,null,"""VIAL""",null,"""0""","""Send Vial""","""1""","""VIAL""","""1""","""VIAL""","""IV"""
"""1389741""","""21""","""109451""","""217847""","""2134-09-13 00:…","""2134-09-16 00:…","""ADDITIVE""","""Calcium Glucon…",null,null,"""CAGLMEQ""","""001356""","""00517391025""","""0.465 mEq / 1 …","""7.5""","""mEq""","""16.129""","""ml""","""DIALYS"""
"""1389739""","""21""","""109451""","""217847""","""2134-09-13 00:…","""2134-09-16 00:…","""MAIN""","""Normocarb""",null,null,"""NORMOCARB""",null,"""0""","""240 mL Vial""","""240""","""ml""","""1""","""VIAL""","""DIALYS"""
"""1389713""","""21""","""109451""","""217847""","""2134-09-13 00:…","""2134-09-16 00:…","""BASE""","""Sterile Water""",null,null,"""SW3000""","""009825""","""00074797308""","""3000 mL Bag""","""3000""","""ml""","""1""","""BAG""","""DIALYS"""
"""1389740""","""21""","""109451""","""217847""","""2134-09-13 00:…","""2134-09-16 00:…","""ADDITIVE""","""Potassium Chlo…",null,null,"""KCL20I""","""001255""","""00074665106""","""2mEq/mL-10mL""","""7.5""","""mEq""","""3.75""","""ml""","""DIALYS"""


In [178]:
#prescript_itemid =  [row[10] for row in df.iter_rows()]

In [180]:
#len(set(prescript_itemid))

In [181]:
valid_prescript =[]
valid_prescript_unit =[]
dropped_id = []
notfound = []
#  prescript_itemid = [r[0] for r in res]

 #   print("len(prescript_itemid) = ", len(prescript_itemid))
results = np.load('/Users/cgarciay/Desktop/Laval_Master_Computer/research/dataset/res/itemids.npy', allow_pickle=True).tolist()


In [182]:
prescript_itemid = results['prescript']


In [228]:
def val(prescript_itemid, df):
    formulary_drug_cd = prescript_itemid

    # Filter the prescriptions DataFrame based on the formulary_drug_cd and admission_ids
    filtered_prescriptions = df.filter(pl.col('FORMULARY_DRUG_CD') == formulary_drug_cd)
    # Group the filtered DataFrame by dose_unit_rx and count the occurrences
    result = filtered_prescriptions.groupby('DOSE_UNIT_RX').count()
    r = result.sort(by= "count", descending=True)
    return (prescript_itemid,list(r["count"]))

In [234]:
results = [val(row,df) for row in prescript_itemid[1:]]

In [235]:
np.save('/Users/cgarciay/Desktop/Laval_Master_Computer/research/dataset/res/filtered_prescript_raw.npy', {'raw': results})

In [260]:
x =results[1]

In [273]:
for x in results:
    i, outputunits = x[0], x[1]
    total = sum(outputunits)
    if total ==0:
        outputunits=[0]
        percentage = 0
    else:
        percentage = float(outputunits[0]) / total *100
    
    valid_prescript.append(i)
    valid_prescript_unit.append(outputunits[0])
    

In [280]:
print(valid_prescript, valid_prescript_unit)
np.save('/Users/cgarciay/Desktop/Laval_Master_Computer/research/dataset/res/filtered_prescript.npy',{'id':valid_prescript,'unit':valid_prescript_unit})
print('saved!')

['REPA2', 'PFNS2MLSYR', 'REPA2', 'PFNS2MLSYR', 'REPA2', 'PFNS2MLSYR', 'CEFO2F', 'MAXIEO', 'TRAZ100', 'ALBU17H', 'NAVA250I', 'NEOVITE', 'VALG450SM', 'CYCL1I', 'AMOX250L', 'CIPROBASE', 'SOLN300I', 'CLOP01L', 'TRAN100I', 'FENT2.5I', 'THEO400TR', 'CEFA500I', 'LUGOL', 'OLOP.1ES', 'SIRO0.5', 'CELE100', 'CLOFPLACIND', 'ACET650L', 'PRIM1I', 'PENI2F', 'ACDA500', 'EPI30I', 'LIND1S', 'THIA200IND', 'DRAM50', 'ZAFI20', 'DICL50', 'DEME300', 'LEV500', 'RIFA550', 'PRAZ5', 'VENF25', 'NIAC500', 'ADACEL', 'AMIK500', 'PHOS250', 'ASCO250I', 'KETA100/50NS', 'ATOR10', 'CAPT50', 'RIS1L', 'CACL1000I', 'TACRSUSPBN', 'ASPARSYR', 'MAAL148', 'CALG500', 'PERP2', 'ACYCIND', 'PROM25', 'SYRINGENS', 'PAMI30I', 'GLUCTAB', 'MAG2PMLF', 'PROM25R', 'NS250G', 'MAG20/500', 'PHEN5N', 'VALP250L', 'OSEL6L', 'HEPVAC', 'CLOP0.5L', 'TRMP100', 'TIAZ360', 'AZA100I', 'HYDRNEO10L', 'FTY7201.25IND', 'DRON25', 'CSA25', 'ETHO250', 'ONDA8ODT', 'TACR5', 'THOR25I', 'TEGR200', 'VENL375XR', 'PROC500I', 'ISON300', 'DIGO125L', 'GUAI10', 'MBLHCV1

In [270]:
percentage = float(outputunits[0]) / total *100
percentage

75.0

In [267]:
outputunits[0]

3

In [247]:
total = 0
for o in outputunits:
    total += o[0]
if(total == 0 ): 
#         print("\n**PRES DROPPED "+str(i))
    notfound.append(i)
    for o in outputunits:
        print(o)
    print('')


TypeError: 'int' object is not subscriptable

'REPA2'

In [183]:
for x in results:
    i, outputunits = x[0], x[1]
    # check if medicine is never used by anybody, then discard it.
    total = 0
    for o in outputunits:
        total += o[1]
    if(total == 0 ): 
#         print("\n**PRES DROPPED "+str(i))
        notfound.append(i)
        for o in outputunits:
            print(o)
        print('')
        continue
        
    # calculate the percentage of observation of main unit.
    percentage = float(outputunits[0][1]) / total *100.
    if(percentage < 90): 
        # never drop the list A
#         if(i in manual_valid):
#             print("\n\n****PRES NOT DROPPED "+str(i) + " : " + "{:.2f}".format(percentage) + " : " + str(len(outputunits))+" : "+ str(outputunits)+"\n")
        
        # drop it !
#         else:
            dropped_id.append(i)
            print("\n\n****PRES DROPPED "+str(i) + " : " + "{:.2f}".format(percentage) + " : " + str(len(outputunits))+" : "+ str(outputunits)+"\n")
            continue
#     print("PRES "+str(i) + " : " + "{:.2f}".format(percentage) + " : " + str(len(outputunits))+" : "+ str(outputunits))
    
    # keep it and also save the main unit of it.
    valid_prescript.append(i)
    valid_prescript_unit.append(outputunits[0][0])

IndexError: string index out of range

In [176]:
len(prescript_itemid)

1000

In [161]:
prescript_itemid

[nan,
 'REPA2',
 'PFNS2MLSYR',
 'CEFO2F',
 'MAXIEO',
 'TRAZ100',
 'ALBU17H',
 'NAVA250I',
 'NEOVITE',
 'VALG450SM',
 'CYCL1I',
 'AMOX250L',
 'CIPROBASE',
 'SOLN300I',
 'CLOP01L',
 'TRAN100I',
 'FENT2.5I',
 'THEO400TR',
 'CEFA500I',
 'LUGOL',
 'OLOP.1ES',
 'SIRO0.5',
 'CELE100',
 'CLOFPLACIND',
 'ACET650L',
 'PRIM1I',
 'PENI2F',
 'ACDA500',
 'EPI30I',
 'LIND1S',
 'THIA200IND',
 'DRAM50',
 'ZAFI20',
 'DICL50',
 'DEME300',
 'LEV500',
 'RIFA550',
 'PRAZ5',
 'VENF25',
 'NIAC500',
 'ADACEL',
 'AMIK500',
 'PHOS250',
 'ASCO250I',
 'KETA100/50NS',
 'ATOR10',
 'CAPT50',
 'RIS1L',
 'CACL1000I',
 'TACRSUSPBN',
 'ASPARSYR',
 'MAAL148',
 'CALG500',
 'PERP2',
 'ACYCIND',
 'PROM25',
 'SYRINGENS',
 'PAMI30I',
 'GLUCTAB',
 'MAG2PMLF',
 'PROM25R',
 'NS250G',
 'MAG20/500',
 'PHEN5N',
 'VALP250L',
 'OSEL6L',
 'HEPVAC',
 'CLOP0.5L',
 'TRMP100',
 'TIAZ360',
 'AZA100I',
 'HYDRNEO10L',
 'FTY7201.25IND',
 'DRON25',
 'CSA25',
 'ETHO250',
 'ONDA8ODT',
 'TACR5',
 'THOR25I',
 'TEGR200',
 'VENL375XR',
 'PROC500I',
 

In [149]:
df.shape

(1637489, 19)

In [133]:
df_filtered.shape

(19993, 117)

In [134]:
df_filtered["LOS_right_isnull"]

ColumnNotFoundError: LOS_right_isnull

In [131]:
df_filtered.with_columns(pl.all().is_null().suffix("_isnull")).count() 

ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,COSTCENTER,CHARTDATE,CPT_CD,CPT_NUMBER,CPT_SUFFIX,TICKET_ID_SEQ,SECTIONHEADER,SUBSECTIONHEADER,DESCRIPTION,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,SEQ_NUM,ICD9_CODE,ICUSTAY_ID,…,DIAGNOSIS_right_isnull,HOSPITAL_EXPIRE_FLAG_right_isnull,HAS_CHARTEVENTS_DATA_right_isnull,SUBMIT_WARDID_isnull,SUBMIT_CAREUNIT_isnull,CURR_WARDID_right_isnull,CURR_CAREUNIT_right_isnull,CALLOUT_WARDID_isnull,CALLOUT_SERVICE_isnull,REQUEST_TELE_isnull,REQUEST_RESP_isnull,REQUEST_CDIFF_isnull,REQUEST_MRSA_isnull,REQUEST_VRE_isnull,CALLOUT_STATUS_isnull,CALLOUT_OUTCOME_isnull,DISCHARGE_WARDID_isnull,ACKNOWLEDGE_STATUS_isnull,CREATETIME_isnull,UPDATETIME_isnull,ACKNOWLEDGETIME_isnull,OUTCOMETIME_isnull,FIRSTRESERVATIONTIME_isnull,CURRENTRESERVATIONTIME_isnull,SEQ_NUM_right_isnull,ICD9_CODE_right_isnull,TRANSFERTIME_isnull,PREV_SERVICE_isnull,CURR_SERVICE_isnull,DBSOURCE_right_isnull,FIRST_CAREUNIT_isnull,LAST_CAREUNIT_isnull,FIRST_WARDID_isnull,LAST_WARDID_isnull,INTIME_right_isnull,OUTTIME_right_isnull,LOS_right_isnull
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str,str,str,str,str,list[str],list[str],list[str],…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
569917021,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1503,18470,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0,0,0,8703,8703,8703,8703,8703,8703,8703,8703,8703,8703,8703,8703,8703,8703,8703,8703,8703,8703,8703,8703,8703,2600,2600,26,26,26,1147,1147,1147,1147,1147,1147,1147,1147


In [119]:
df_filtered.write_csv("data/df_filtered.csv.gz ")

ComputeError: CSV format does not support nested data

In [ ]:
list_ar = ['dataset/CALLOUT.csv.gz','dataset/SERVICES.csv.gz','dataset/CPTEVENTS.csv.gz',]

# Obtener layout dopplergangar

In [119]:
merged_df9.to_csv("data/first_input.csv")

In [158]:
df = merged_df9.copy()

In [157]:
cols = ["SUBJECT_ID","visit","DIAGNOSIS","MARITAL_STATUS","WHITE","INSURANCE","DRUG_services_y","DOSE_VAL_RX_services_y","GENDER","DOB","DOD","EXPIRE_FLAG","SHORT_TITLE_services_x"]

In [201]:
merged_df9["LOS_services_y"].value_counts()

Series([], Name: LOS_services_y, dtype: int64)

In [120]:
pd.set_option('display.max_columns', None)
merged_df9

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,SUBMIT_CAREUNIT_callout,CURR_CAREUNIT_callout,CALLOUT_SERVICE_callout,REQUEST_MRSA_callout,REQUEST_VRE_callout,CALLOUT_STATUS_callout,CALLOUT_OUTCOME_callout,ACKNOWLEDGE_STATUS_callout,CREATETIME_callout,UPDATETIME_callout,ACKNOWLEDGETIME_callout,OUTCOMETIME_callout,TRANSFERTIME_services_x,PREV_SERVICE_services_x,CURR_SERVICE_services_x,TRANSFERTIME_services_y,PREV_SERVICE_services_y,CURR_SERVICE_services_y,SEQ_NUM_services,ICD9_CODE_services_x,SHORT_TITLE_services_x,DRG_TYPE_services,DESCRIPTION_services,DRG_SEVERITY_services,DRG_MORTALITY_services,ICUSTAY_ID_services_x,STARTDATE_services_x,ENDDATE_services_x,DRUG_TYPE_services_x,DRUG_services_x,DRUG_NAME_POE_services_x,DRUG_NAME_GENERIC_services_x,FORMULARY_DRUG_CD_services_x,PROD_STRENGTH_services_x,DOSE_VAL_RX_services_x,DOSE_UNIT_RX_services_x,FORM_VAL_DISP_services_x,FORM_UNIT_DISP_services_x,ROUTE_services_x,ICUSTAY_ID_services_y,STARTDATE_services_y,ENDDATE_services_y,DRUG_TYPE_services_y,DRUG_services_y,DRUG_NAME_POE_services_y,DRUG_NAME_GENERIC_services_y,FORMULARY_DRUG_CD_services_y,PROD_STRENGTH_services_y,DOSE_VAL_RX_services_y,DOSE_UNIT_RX_services_y,FORM_VAL_DISP_services_y,FORM_UNIT_DISP_services_y,ROUTE_services_y,ICD9_CODE_services_y,SHORT_TITLE_services_y,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,ICUSTAY_ID_services_x,FIRST_CAREUNIT_services,LAST_CAREUNIT_services,INTIME_services_x,OUTTIME_services_x,LOS_services_x,ICUSTAY_ID_services_y,EVENTTYPE_services,PREV_CAREUNIT_services,CURR_CAREUNIT_services,INTIME_services_y,OUTTIME_services_y,LOS_services_y
0,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,2082-07-17 00:00:00,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,2082-07-17 00:00:00,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,33,34,115799,2186-07-18 16:46:00,2186-07-20 16:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,CHEST PAIN\CATH,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,1886-07-18 00:00:00,2192-01-30 00:00:00,NaN,2192-01-30 00:00:00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,34,34,144319,2191-02-23 05:23:00,2191-02-25 20:20:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,2191-02-23 04:23:00,2191-02-23 07:25:00,BRADYCARDIA,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,1886-07-18 00:00:00,2192-01-30 00:00:00,NaN,2192-01-30 00:00:00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,36,36,182104,2131-04-30 07:15:00,2131-05-08 14:00:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,CORONARY

In [168]:
merged_df9 = df.sort_values('ADMITTIME')

# Agrupa por 'subject_id' y cuenta las visitas para cada 'subject_id'
merged_df9['visitas'] = merged_df9.groupby('SUBJECT_ID').cumcount() + 1




In [198]:
df.visitas.value_counts()

1     7537
2     7537
3     2377
4     1035
5      527
6      281
7      168
8      117
9       86
10      60
11      46
12      33
13      25
14      20
15      16
16      15
17      13
18      10
19      10
20       9
21       8
22       7
23       6
24       5
25       4
26       4
27       4
28       4
29       4
30       4
31       4
34       3
33       3
32       3
35       1
36       1
37       1
38       1
39       1
40       1
41       1
42       1
Name: visitas, dtype: int64

In [185]:
import pickle

file = "data_train.npz"
data = np.load(file)

# Access the individual arrays inside the .npz file
data.files

['data_gen_flag', 'data_attribute', 'data_feature']

In [190]:
data['data_feature'].shape

(50000, 550, 1)

In [192]:
data['data_attribute'].shape

(50000, 14)

In [206]:
data['data_attribute'][100]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0], dtype=uint8)

In [210]:
data['data_gen_flag'][2]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [ ]:
550 view

In [193]:
data['data_gen_flag']

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [194]:
data['data_feature']

array([[[-0.33379488],
        [-0.319665  ],
        [-0.31494249],
        ...,
        [-0.24167157],
        [-0.25404153],
        [-0.2666114 ]],

       [[-0.69234718],
        [-0.69950852],
        [-0.74449986],
        ...,
        [-0.70716413],
        [-0.76926038],
        [-0.70716413]],

       [[-0.29111053],
        [-0.29863301],
        [-0.3462496 ],
        ...,
        [-0.53179372],
        [-0.56371558],
        [-0.59636507]],

       ...,

       [[-0.72426904],
        [-0.63025092],
        [-0.6792777 ],
        ...,
        [-0.63847411],
        [-0.68562013],
        [-0.70716413]],

       [[-0.37124026],
        [-0.36489783],
        [-0.35518243],
        ...,
        [-0.19127963],
        [-0.18791801],
        [-0.21271533]],

       [[-0.38340157],
        [-0.36028113],
        [-0.41320172],
        ...,
        [-0.55737315],
        [-0.50069077],
        [-0.55333772]]])

In [182]:
data

In [169]:
df = merged_df9.copy()

In [170]:
subject_data = df[df["SUBJECT_ID"] == id_]

In [175]:
subject_data.sort_values(by="visitas")

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,SUBMIT_CAREUNIT_callout,CURR_CAREUNIT_callout,CALLOUT_SERVICE_callout,REQUEST_MRSA_callout,REQUEST_VRE_callout,CALLOUT_STATUS_callout,CALLOUT_OUTCOME_callout,ACKNOWLEDGE_STATUS_callout,CREATETIME_callout,UPDATETIME_callout,ACKNOWLEDGETIME_callout,OUTCOMETIME_callout,TRANSFERTIME_services_x,PREV_SERVICE_services_x,CURR_SERVICE_services_x,TRANSFERTIME_services_y,PREV_SERVICE_services_y,CURR_SERVICE_services_y,SEQ_NUM_services,ICD9_CODE_services_x,SHORT_TITLE_services_x,DRG_TYPE_services,DESCRIPTION_services,DRG_SEVERITY_services,DRG_MORTALITY_services,ICUSTAY_ID_services_x,STARTDATE_services_x,ENDDATE_services_x,DRUG_TYPE_services_x,DRUG_services_x,DRUG_NAME_POE_services_x,DRUG_NAME_GENERIC_services_x,FORMULARY_DRUG_CD_services_x,PROD_STRENGTH_services_x,DOSE_VAL_RX_services_x,DOSE_UNIT_RX_services_x,FORM_VAL_DISP_services_x,FORM_UNIT_DISP_services_x,ROUTE_services_x,ICUSTAY_ID_services_y,STARTDATE_services_y,ENDDATE_services_y,DRUG_TYPE_services_y,DRUG_services_y,DRUG_NAME_POE_services_y,DRUG_NAME_GENERIC_services_y,FORMULARY_DRUG_CD_services_y,PROD_STRENGTH_services_y,DOSE_VAL_RX_services_y,DOSE_UNIT_RX_services_y,FORM_VAL_DISP_services_y,FORM_UNIT_DISP_services_y,ROUTE_services_y,ICD9_CODE_services_y,SHORT_TITLE_services_y,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,ICUSTAY_ID_services_x,FIRST_CAREUNIT_services,LAST_CAREUNIT_services,INTIME_services_x,OUTTIME_services_x,LOS_services_x,ICUSTAY_ID_services_y,EVENTTYPE_services,PREV_CAREUNIT_services,CURR_CAREUNIT_services,INTIME_services_y,OUTTIME_services_y,LOS_services_y,visitas
37,116,109,183350,2137-11-04 19:36:00,2137-11-21 18:13:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Government,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2137-11-04 12:48:00,2137-11-04 19:37:00,HYPERTENSIVE EMERGENCY,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,2117-08-07 00:00:00,2142-08-30 00:00:00,2142-08-30 00:00:00,2142-08-30 00:00:00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
38,117,109,128755,2138-04-16 17:44:00,2138-04-19 16:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2138-04-16 10:55:00,2138-04-16 19:14:00,HTN CRISIS,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,2117-08-07 00:00:00,2142-08-30 00:00:00,2142-08-30 00:00:00,2142-08-30 00:00:00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
39,118,109,164029,2140-01-19 13:25:00,2140-01-21 13:25:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2140-01-19 10:04:00,2140-01-19 15:01:00,HYPERTENSION,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,2117-08-07 00:00:00,2142-08-30 00:00:00,2142-08-30 00:00:00,2142-08-30 00:00:00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
40,119,109,108375,2140-02-02 02:13:00,2140-02-02 16:25:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,ENGL,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2140-02-01 22:03:00,2140-02-02 03:00:00,HYPERTENSIVE URGENCY,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,2117-08-07 00:00:00,2142-08-30 00:00:00,214

In [ ]:
import pandas as pd

# Assuming your DataFrame is called "df"
subject_id_column = "SUBJECT_ID"

# Get unique subject IDs
unique_subject_ids = df[subject_id_column].unique()

# Initialize lists to store the time series and data features
time_series_list = []
data_feature_list = []

# Iterate over unique subject IDs
for subject_id in unique_subject_ids:
    # Filter the DataFrame for the current subject ID
    subject_data = df[df[subject_id_column] == subject_id]
    
    # Extract the time series and data features from the filtered DataFrame
    time_series = subject_data["time_series_column"].tolist()  # Replace "time_series_column" with the actual column name
    data_feature = subject_data["data_feature_column"].tolist()  # Replace "data_feature_column" with the actual column name
    
    # Append the time series and data features to the respective lists
    time_series_list.append(time_series)
    data_feature_list.append(data_feature)
